### Feature Engineering

In [0]:
import pandas as pd
import nltk
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Load Data

In [0]:
train = pd.read_csv('train_data.csv')

Create feature dataframe

In [0]:
train_features = pd.DataFrame()

## Define Functions

In [21]:

stop_words = set(stopwords.words('english'))

def same_words_fraction(entry):
    w1 = set()
    w2 = set()
    
    if (entry['question1']):
        w1 = set(map(lambda word: word.lower().strip(), entry['question1'].split(' ')))
    if (entry['question2']):
        w2 = set(map(lambda word: word.lower().strip(), entry['question2'].split(' ')))

    return len(w1 & w2)/(len(w1) + len(w2))
  
def get_length_features(data):
    data['q1_len'] = data.question1.apply(lambda x: len(str(x)))
    data['q2_len'] = data.question2.apply(lambda x: len(str(x)))
    data['len_diff'] = data.q1_len - data.q2_len

    return data
  
def get_jaccard_sim(entry):
    q1 = entry['question1']
    q2 = entry['question2']

    a = set(q1.split()) 
    b = set(q2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))
  
def get_weight(count,eps=10000,min_count=2):
    if count<min_count:
        return 0
    else:
        return 1.0/(count+eps)

eps=5000
words=(" ".join(train_qs)).lower().split()
counts=Counter(words)
weights={word:get_weight(count) for word,count in counts.items()}


def tfidf_same_words_fraction(row):
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stop_words:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stop_words:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    
    shared_weights = [weights.get(w, 0) for w in q1words.keys() if w in q2words] + [weights.get(w, 0) for w in q2words.keys() if w in q1words]
    total_weights = [weights.get(w, 0) for w in q1words] + [weights.get(w, 0) for w in q2words]
        
    R = (np.sum(shared_weights)+0.0) / np.sum(total_weights)
    return R


NameError: ignored

## Features

In [19]:
train_features['same_words_fraction'] = train.apply(same_words_fraction, axis=1, raw=True)
train_features['tfidf_same_words_fraction'] = train.apply(tfidf_same_words_fraction, axis=1, raw=True)

length_features = get_length_features(train)
train_features['q1_len'] = length_features['q1_len']
train_features['q2_len'] = length_features['q2_len']
train_features['len_diff'] = length_features['len_diff']
train_features['jacard_similarity'] = train.apply(get_jaccard_sim, axis=1, raw=True)

NameError: ignored